In [7]:
# prompt: 구글드라이브 연동해줘

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/03. 데이터 불러오기

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/03. 데이터 불러오기


### 일반 파일 불러오기 (로컬)

In [8]:
import pandas as pd

In [10]:
csData = pd.read_csv("../dataset/customerdata.csv")

### github 공유된 파일 불러오는것도 가능하다

In [11]:
carDataPath = "https://raw.githubusercontent.com/hyokwan/python-lecture/refs/heads/master/dataset/cars.csv"

In [13]:
carsDf = pd.read_csv(carDataPath)

### 공공데이터포털 데이터 불러오기

In [15]:
pirPath = "https://www.data.go.kr/cmm/cmm/fileDownload.do?atchFileId=FILE_000000002999024&fileDetailSn=1&dataNm=%ED%95%9C%EA%B5%AD%EC%A3%BC%ED%83%9D%EA%B8%88%EC%9C%B5%EA%B3%B5%EC%82%AC_%EC%A3%BC%ED%83%9D%EA%B8%88%EC%9C%B5%EA%B4%80%EB%A0%A8%20%EC%A7%80%EC%88%98_20160101"
pirDf = pd.read_csv( pirPath, encoding = "ms949" )

### 데이터베이스 데이터 불러오기

In [ ]:
## 08:05 스타트